<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-librerias" data-toc-modified-id="Cargamos-librerias-1">Cargamos librerias</a></span><ul class="toc-item"><li><span><a href="#EDA:-Exploratory-data-analysis" data-toc-modified-id="EDA:-Exploratory-data-analysis-1.1">EDA: Exploratory data analysis</a></span></li></ul></li></ul></div>

# Cargamos librerias

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import scipy as sc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display
import sys

## EDA: Exploratory data analysis

In [ ]:
!wget

In [4]:
df=pd.read_csv('datos/iris.csv')
print(df.shape)
df.head(1)

FileNotFoundError: File b'datos/iris.csv' does not exist